# 1. Vector Database Generation
This code generates the vector database for a PDF document. For this application, a car manual.
This code should be run only once, since the results are save in disk.

Authors:
- Luis Bernardo Hernandez Salinas
- Juan R. Terven

In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

## Set API key, embedding and vector database path

In [4]:
client = os.environ['OPENAI_API_KEY']
embedding_dimensions = 3072 #1536  # 3072

# OpenAI embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=embedding_dimensions)

# Vector dataset
vehicle_name = "kia_sorento"
manual_file = "car_manuals/2019-Kia-Sorento-Owners-Manual.pdf"
vectordb_directory = f'vector_database_{vehicle_name}_{embedding_dimensions}'

print(f"Output directory: {vectordb_directory}")

Output directory: vector_database_kia_sorento_3072


## Load PDF

In [5]:
loader = PyPDFLoader(manual_file)

# load pdf pagesÇ
pages = loader.load()
print(f"The document has {len(pages)} pages")

The document has 585 pages


In [6]:
# Lets check one page
pages[100]

Document(metadata={'source': 'car_manuals/2019-Kia-Sorento-Owners-Manual.pdf', 'page': 100}, page_content="41 3Features of your vehicle\nUnlocking\nUsing the door handle button\n1. Carry the smart key.2. Press the button of the driver's\noutside door handle.\n3. The driver’s door will unlock. The\nhazard warning lights will blink and\nthe chime will sound two times.\n4. Press the button twice within 4\nseconds and all doors and the lift-gate will unlock and the hazardwarning lights will blink and thechime will sound two times.\n✽✽NOTICE\n• The button will only operate when\nthe smart key is within 28~40in.\n(0.7~1m) from the outside doorhandle.\n• When the smart key is recognized in\nthe area of 28~40in. (0.7~1m) fromthe front outside door handle, otherpeople can also open the doors.\n• After unlocking the driver’s door\nor all doors, the door(s) will lockautomatically unless the door isopened.Using the button on the smart key\n1. Press the unlock button(2) of the\nsmart key.\n2. The d

## Documents Splitting

In [7]:
# RecursiveCharacterTextSplitter with overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,  # chunk size in characters
    chunk_overlap = 150 # Caracteres de solapamiento entre segmentos consecutivos.
)

# split documents
splits = text_splitter.split_documents(pages)

print(f"Generated {len(splits)} splits")

Generated 646 splits


## Create Vector database

In [8]:
# Create vector database from the embedding of the documents splits 
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
    persist_directory=vectordb_directory
)

print(f"Saved {vectordb._collection.count()} collections in vector database")

Saved 646 collections in vector database
